In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv


In [73]:
!rm train_targets_folds.csv
!wget https://github.com/Mayukhdeb/moa-classification-with-model-blending/raw/main/folds/train_targets_folds.csv

--2020-10-06 07:49:26--  https://github.com/Mayukhdeb/moa-classification-with-model-blending/raw/main/folds/train_targets_folds.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Mayukhdeb/moa-classification-with-model-blending/main/folds/train_targets_folds.csv [following]
--2020-10-06 07:49:26--  https://raw.githubusercontent.com/Mayukhdeb/moa-classification-with-model-blending/main/folds/train_targets_folds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10182351 (9.7M) [text/plain]
Saving to: ‘train_targets_folds.csv’

train_targets_folds 100%[===================>]   9.71M  9.85MB/s    in 1.0s    

2020-10-06 07:49:28 (9.85 

In [74]:
!pip install iterative-stratification

In [75]:
pd.read_csv("./train_targets_folds.csv").head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_95029ba69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,id_937bf8e8f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2,id_59e0b53af,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,id_1b0b65599,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,id_1d650ed9d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [76]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

import numpy as np 
import pandas as pd 

from IPython.display import clear_output
import random 
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm, tnrange
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

plt.rcParams['figure.figsize'] = 15, 7

CGREEN  = '\33[32m'
CBLUE =  '\033[34m'
CRED = '\033[1;31m'
CEND  = '\33[0m'

def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print("seed kar diya")

In [77]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
    
device


'cuda'

In [78]:

def preprocess(df, select_features):
    df = df.copy()
    top_features = [ 0, 1, 2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]
    all_columns = df.columns

    if select_features == True:
        df=df[all_columns[top_features]]
    else:
        pass
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({ 24: 0, 48: 1, 72: 2})
    
    return df

In [79]:
class MoaDataset(Dataset):
    def __init__(self,features,targets):
        
        self.features  = features
        self.targets = targets
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x": torch.tensor(self.features[item, :], dtype=torch.float),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float)
        }

In [80]:

class Engine:
      def __init__(self, model, optimizer, device, scheduler):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler

      @staticmethod
      def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

      def train(self, data_loader):
        self.model.train()
        final_loss=0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            
            self.optimizer.step()
            
            final_loss += loss.item()
        self.scheduler.step(1.)
        return(final_loss / len(data_loader))


      def evaluate(self, data_loader):
        self.model.eval()
        final_loss=0
        for data in data_loader:
            #self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            #loss.backward()
            #self.optimizer.step()
            final_loss += loss.item()
        return(final_loss / len(data_loader))

In [81]:
def run_training(model_class ,fold, save_model=True, select_features = False, model_id = 0):
    EPOCHS = 100
    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = preprocess(df, select_features = select_features)
    targets_df = pd.read_csv('./train_targets_folds.csv')
    
    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns
    

    df = df.merge(targets_df, on="sig_id", how="left")
    print(df.shape)
    df = df[df.kfold != "hold"]
    print(df.shape)

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)

    
    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()
    
    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()
    
    train_dataset = MoaDataset(features = x_train, targets=y_train)
    valid_dataset = MoaDataset(features = x_val, targets=y_val)
    
    train_loader = DataLoader(
        train_dataset, batch_size=128, shuffle=True
    )
    
    val_loader = DataLoader(
        valid_dataset, batch_size=1024, shuffle=True
    )
    
    model = model_class()
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), 
                                       lr = 0.004299882049752947, 
                                       weight_decay=2e-5)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.1, ## wooo hoo
                                                 patience=7, ## was 3 for 158 
                                                 eps=1e-4, 
                                                 verbose=True)
    criterion = nn.BCEWithLogitsLoss()
    eng = Engine(model, optimizer, device='cuda', scheduler = scheduler)
    
    best_loss = 999
    early_stop_iter = 15
    early_stop_count=0

    for epoch in tnrange(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(val_loader)
        #print("train_loss:", train_loss, "val_loss:", valid_loss)
        if valid_loss<best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), "model_"+str(fold + model_id) +".pth")
                pass
        else:
            early_stop_count +=1
            
        if early_stop_count>early_stop_iter:
            break
                    
    return(best_loss)


In [90]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(785)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(785, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, 206))
        
        self.scalar = 2.0
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)* self.scalar
        return x

In [91]:
best_losses = []
num_folds = 7
for fold in range(0, num_folds):
    print(best_losses)
    
    best_losses.append(run_training(model_class = Model, fold=fold, select_features = True))

[]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014727714161078135]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014727714161078135, 0.014860187967618307]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014727714161078135, 0.014860187967618307, 0.014923783640066782]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014727714161078135, 0.014860187967618307, 0.014923783640066782, 0.014638728462159634]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014727714161078135, 0.014860187967618307, 0.014923783640066782, 0.014638728462159634, 0.014709356861809889]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014727714161078135, 0.014860187967618307, 0.014923783640066782, 0.014638728462159634, 0.014709356861809889, 0.014776303743322691]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.



In [93]:
print(best_losses)  ## better than old model
np.array(best_losses).mean()

[0.014727714161078135, 0.014860187967618307, 0.014923783640066782, 0.014638728462159634, 0.014709356861809889, 0.014776303743322691, 0.015126216846207777]


0.014823184526037602

In [88]:

class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(875)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(875, 2048))  ## was 948 and 2 layers ofr best yet 128
        

        
        self.batch_norm3 = nn.BatchNorm1d(2048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(2048, 206))
    
        self.scalar = 2.0
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))

        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)*self.scalar
        
        return x

In [89]:
best_losses_2 = []
num_folds = 7
for fold in range(num_folds):
    print(best_losses_2)
    best_losses_2.append(run_training(model_class = Model_2, fold=fold, select_features = False, model_id = 100))

[]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014842134589950243]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014842134589950243, 0.014941730226079622]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014842134589950243, 0.014941730226079622, 0.014951237787803015]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014842134589950243, 0.014941730226079622, 0.014951237787803015, 0.014706349310775598]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014842134589950243, 0.014941730226079622, 0.014951237787803015, 0.014706349310775598, 0.014866584911942482]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014842134589950243, 0.014941730226079622, 0.014951237787803015, 0.014706349310775598, 0.014866584911942482, 0.014822808715204397]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.



In [92]:
print(best_losses_2)  ## better than old model
np.array(best_losses_2).mean()

[0.014842134589950243, 0.014941730226079622, 0.014951237787803015, 0.014706349310775598, 0.014866584911942482, 0.014822808715204397, 0.01521244024236997]


0.014906183683446475

In [ ]:
!zip -r models.zip *.pth 

In [94]:
def load_fold(fold, select_features = False):

    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = preprocess(df, select_features = select_features)
    targets_df = pd.read_csv('./train_targets_folds.csv')

    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns


    df = df.merge(targets_df, on="sig_id", how="left")

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)

    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()

    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()

    return x_train, y_train, x_val, y_val

class blend():
    def __init__(self,all_preds_np):
        
        self.all_preds = all_preds_np
           
    def predict(self, weights):
        self.weights = weights
        final_pred = np.zeros_like(self.all_preds[0])
        
        for i in range(len(self.all_preds)):
            final_pred += self.all_preds[i] * self.weights[i]
            
        final_pred = final_pred/self.weights.sum()
        
        return final_pred


def inference_fn(model, test_features, device):

    results = np.zeros([test_features.shape[0], 206])
    test_features_tensor = torch.tensor(test_features).float().to(device)
    rows = model(test_features_tensor).sigmoid().detach().cpu().numpy()
    results = rows 

    return results 

class pytorch_model():
    def __init__(self, model_class, model_path, device):
        
        self.model_class = model_class
        self.model_path = model_path
        self.device = device
        
        self.model = self.model_class()
        if self.device == "cuda":
            self.model.load_state_dict(torch.load(self.model_path))
        else:
            self.model.load_state_dict(torch.load(self.model_path, map_location=torch.device('cpu') ))

        self.model.to(self.device)
        self.model.eval()
        
    def predict(self, x):
        
        pred = self.model(x)
        
        return pred 
    
    def __call__(self, x):
        return self.predict(x)


def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

In [95]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

all_preds = []
for i in range(7):
    m1  = pytorch_model(Model, "model_"+ str(i) +".pth", device)
    p1 = inference_fn(m1, x_val, device)
    l1 = log_loss_metric(y_val, p1)
    print(l1)
    all_preds.append(p1)

0.0146674760931631
0.014694759783387606
0.014668408477570392
0.014673047689170288
0.014677514131652495
0.014717222165351774
0.01471859346547459


In [96]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = False)

for i in range(7):
    m2  = pytorch_model(Model_2, "model_"+ str(i+100) +".pth", device)
    p2 = inference_fn(m2, x_val, device)
    l2 = log_loss_metric(y_val, p2)
    print(l2)
    all_preds.append(p2)

0.014814834758875468
0.014761426655414052
0.014697168109893402
0.014759606921275098
0.014718271530778157
0.014748115941998955
0.014775360445550726


## Now train an XGBoost model and blend with optuna  **{ON HOLD RN}**

[have a look pls](https://www.kaggle.com/fchmiel/xgboost-baseline-multilabel-classification)

[emergency source for 0.0185 something](https://www.kaggle.com/ksouriazer/the-power-of-blend)

In [ ]:
# from xgboost import XGBClassifier
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.pipeline import Pipeline


In [ ]:
# classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))


# clf = Pipeline([
#                 ('classify', classifier)
#                ])

# params = {'classify__estimator__colsample_bytree': 0.6522,
#           'classify__estimator__gamma': 3.6975,
#           'classify__estimator__learning_rate': 0.0503,
#           'classify__estimator__max_delta_step': 2.0706,
#           'classify__estimator__max_depth': 10,
#           'classify__estimator__min_child_weight': 31.5800,
#           'classify__estimator__n_estimators': 166,
#           'classify__estimator__subsample': 0.8639
#          }

# _ = clf.set_params(**params)

In [ ]:
# DATA_DIR = '/kaggle/input/lish-moa/'
# test = pd.read_csv(DATA_DIR + 'test_features.csv')
# x_test = test.iloc[:,1:].to_numpy()


# num_folds = 7

# for fold in range(num_folds):
#     df = pd.read_csv("../input/lish-moa/train_features.csv")
#     df = preprocess(df, select_features = True)

#     targets_df = pd.read_csv('./train_targets_folds.csv')

#     feature_columns = df.drop("sig_id", axis=1).columns
#     target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns


#     df = df.merge(targets_df, on="sig_id", how="left")
#     print(df.shape)
#     df = df[df.kfold != "hold"]
#     print(df.shape)

#     train_df = df[df.kfold != str(fold)].reset_index(drop=True)
#     valid_df = df[df.kfold == str(fold)].reset_index(drop=True)


#     x_train = train_df[feature_columns].to_numpy()
#     y_train = train_df[target_columns].to_numpy()

#     x_val = valid_df[feature_columns].to_numpy()
#     y_val = valid_df[target_columns].to_numpy()
    
#     print("fitting...")
#     clf.fit(x_train, y_train)
    
#     print("done")
#     val_preds = clf.predict_proba(x_val) # list of preds per class
#     val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    
#     print("making preds...")
#     preds = clf.predict_proba(x_test)
#     all_preds.append(preds)
    
#     print(log_loss_metric(y_val, val_preds))

In [97]:
!pip install optuna
clear_output()

In [98]:
import optuna

In [99]:
def scale_data(data, lower, upper):
    
    scaled = np.interp(data, (data.min(), data.max()), (lower, upper))
    return scaled 

def find_loss(params):
    
    weights_np = np.array(list(params.values()))
    blend_boi = blend(all_preds)
    sample_infer = blend_boi.predict(weights_np)
    
    
    loss = log_loss_metric(y_val, sample_infer)
    
    return loss


def objective(trial):

    params = {
        "w1": trial.suggest_uniform("w1", 0.0001, 1.0),
        "w2": trial.suggest_uniform("w2", 0.0001, 1.0),
        "w3": trial.suggest_uniform("w3", 0.0001, 1.0),
        "w4": trial.suggest_uniform("w4", 0.0001, 1.0),
        "w5": trial.suggest_uniform("w5", 0.0001, 1.0),
        "w6": trial.suggest_uniform("w6", 0.0001, 1.0),
        "w7": trial.suggest_uniform("w7", 0.0001, 1.0),
        "w8": trial.suggest_uniform("w8", 0.0001, 1.0),
        "w9": trial.suggest_uniform("w9", 0.0001, 1.0),
        "w10": trial.suggest_uniform("w10", 0.0001, 1.0),
        "w11": trial.suggest_uniform("w11", 0.0001, 1.0),
        "w12": trial.suggest_uniform("w12", 0.0001, 1.0),
        "w13": trial.suggest_uniform("w13", 0.0001, 1.0),
        "w14": trial.suggest_uniform("w14", 0.0001, 1.0)

    }

    loss_ = find_loss(params) 
    return loss_

In [100]:
study = optuna.create_study(direction="minimize")

[I 2020-10-06 08:10:26,191] A new study created in memory with name: no-name-3c81d225-2581-4044-820c-208ce979c7fc


In [101]:
len(all_preds)

14

In [102]:
study.optimize(objective, n_trials=1000)

[I 2020-10-06 08:10:29,107] Trial 0 finished with value: 0.0144849633808865 and parameters: {'w1': 0.3614676300368368, 'w2': 0.3278493333429025, 'w3': 0.4518570038357219, 'w4': 0.9386044334086475, 'w5': 0.7798333263486773, 'w6': 0.293292110920815, 'w7': 0.030015512608556116, 'w8': 0.5398780167421448, 'w9': 0.8643984409346233, 'w10': 0.6088290022021331, 'w11': 0.29323474040722763, 'w12': 0.37445800935930373, 'w13': 0.5607346704138425, 'w14': 0.7069895683784879}. Best is trial 0 with value: 0.0144849633808865.
[I 2020-10-06 08:10:29,132] Trial 1 finished with value: 0.014489028654901518 and parameters: {'w1': 0.26940706977662143, 'w2': 0.035335216742194135, 'w3': 0.5052591577022296, 'w4': 0.6038448005263662, 'w5': 0.9127986202236688, 'w6': 0.7135300993911385, 'w7': 0.8949500493315748, 'w8': 0.9700783298693044, 'w9': 0.8689544656864506, 'w10': 0.22099474501906716, 'w11': 0.32054246844840106, 'w12': 0.7297166425236545, 'w13': 0.08736753999722904, 'w14': 0.12767236242465121}. Best is trial 

[I 2020-10-06 08:10:29,608] Trial 15 finished with value: 0.014482403482132922 and parameters: {'w1': 0.60447089174132, 'w2': 0.5741393391966085, 'w3': 0.0666405493934284, 'w4': 0.9947018680561849, 'w5': 0.433106943897419, 'w6': 0.5688302486485965, 'w7': 0.36703224111667154, 'w8': 0.4149267601870455, 'w9': 0.011156208376766402, 'w10': 0.9237908677492792, 'w11': 0.5413758394469945, 'w12': 0.48118596233799465, 'w13': 0.6687041667877087, 'w14': 0.5107927217525046}. Best is trial 14 with value: 0.014481164926932159.
[I 2020-10-06 08:10:29,662] Trial 16 finished with value: 0.014485151206919804 and parameters: {'w1': 0.9985594414503316, 'w2': 0.7129884170461492, 'w3': 0.23715175677106412, 'w4': 0.7123693668280946, 'w5': 0.7178392096269974, 'w6': 0.6678489740321337, 'w7': 0.2562052967568317, 'w8': 0.6231470425802796, 'w9': 0.0829291963019484, 'w10': 0.6605319297944875, 'w11': 0.681859519549917, 'w12': 0.59684467114549, 'w13': 0.9679992933502741, 'w14': 0.26327339374175773}. Best is trial 14 

[I 2020-10-06 08:10:30,334] Trial 30 finished with value: 0.01448382701543976 and parameters: {'w1': 0.39280777587807547, 'w2': 0.6207688816288975, 'w3': 0.5557970724049658, 'w4': 0.024920990416224142, 'w5': 0.8082410271277581, 'w6': 0.27304728907135795, 'w7': 0.7265321027179856, 'w8': 0.5101877547729051, 'w9': 0.21752087479054893, 'w10': 0.41277257886855284, 'w11': 0.36994579470352806, 'w12': 0.3259366627595931, 'w13': 0.50275771089376, 'w14': 0.494034789604106}. Best is trial 27 with value: 0.014480351524280274.
[I 2020-10-06 08:10:30,382] Trial 31 finished with value: 0.014480608021948062 and parameters: {'w1': 0.5228155587544115, 'w2': 0.40458988816067026, 'w3': 0.7123798913731005, 'w4': 0.9031166290553847, 'w5': 0.7128231882744989, 'w6': 0.032443244844397895, 'w7': 0.518509765956597, 'w8': 0.7244927125808962, 'w9': 0.3947488038266075, 'w10': 0.9181868490640801, 'w11': 0.5830392667261527, 'w12': 0.31838467105323887, 'w13': 0.4203711162689677, 'w14': 0.33113270187973987}. Best is tr

[I 2020-10-06 08:10:31,118] Trial 45 finished with value: 0.014493014416864822 and parameters: {'w1': 0.8871026820591817, 'w2': 0.060632634385578255, 'w3': 0.4523445215647448, 'w4': 0.1532287755987937, 'w5': 0.9953530050300685, 'w6': 0.21040733413111337, 'w7': 0.1586149604765832, 'w8': 0.16044704368725746, 'w9': 0.9343487384611384, 'w10': 0.00036119403492707036, 'w11': 0.02545032583380885, 'w12': 0.0463798270816177, 'w13': 0.31238842754502155, 'w14': 0.033865147315949404}. Best is trial 36 with value: 0.014478389980731192.
[I 2020-10-06 08:10:31,168] Trial 46 finished with value: 0.014480640028638126 and parameters: {'w1': 0.910350668429772, 'w2': 0.1640391391493657, 'w3': 0.5062260500870036, 'w4': 0.07979116632275768, 'w5': 0.9678788690495325, 'w6': 0.3817802040082604, 'w7': 0.16831344022942438, 'w8': 0.18827052927269106, 'w9': 0.8927948819919889, 'w10': 0.8039145629361137, 'w11': 0.07856016062967028, 'w12': 0.017286774398300883, 'w13': 0.3774970958090815, 'w14': 0.1483599039344736}. 

[I 2020-10-06 08:10:31,880] Trial 60 finished with value: 0.014491483618594925 and parameters: {'w1': 0.6255070177964334, 'w2': 0.18524775876245314, 'w3': 0.6610386471635727, 'w4': 0.1001484743133246, 'w5': 0.023849228987476434, 'w6': 0.19789712258912567, 'w7': 0.3322025127974112, 'w8': 0.40756918508067036, 'w9': 0.9901591251582549, 'w10': 0.9416157854614927, 'w11': 0.28180681129523544, 'w12': 0.07202541300133394, 'w13': 0.40274374010080516, 'w14': 0.24112101927499102}. Best is trial 57 with value: 0.014478123497595188.
[I 2020-10-06 08:10:31,936] Trial 61 finished with value: 0.014484538972613939 and parameters: {'w1': 0.9334274496979678, 'w2': 0.27783849523292703, 'w3': 0.49924841139335374, 'w4': 0.18286444251863715, 'w5': 0.33546676013379784, 'w6': 0.39172716818744757, 'w7': 0.1425021146574773, 'w8': 0.14649503363931357, 'w9': 0.9183490215006418, 'w10': 0.47347945917730805, 'w11': 0.01801316478111986, 'w12': 0.02753014108193048, 'w13': 0.32267641238316513, 'w14': 0.29146157659358696

...

[I 2020-10-06 08:11:55,274] Trial 930 finished with value: 0.01447296658074854 and parameters: {'w1': 0.4434648840925809, 'w2': 0.29699459366289477, 'w3': 0.11388199242122385, 'w4': 0.3121959908810461, 'w5': 0.8266808519943127, 'w6': 0.10557367386185411, 'w7': 0.4015428367026968, 'w8': 0.00044069203735935814, 'w9': 0.4954073566919448, 'w10': 0.9142670919871736, 'w11': 0.19330009393381473, 'w12': 0.1366616180086534, 'w13': 0.10800011392372083, 'w14': 0.033119815283298666}. Best is trial 921 with value: 0.014472813578099684.
[I 2020-10-06 08:11:55,408] Trial 931 finished with value: 0.014473047352802393 and parameters: {'w1': 0.448613838808849, 'w2': 0.297057189062461, 'w3': 0.10881949150700639, 'w4': 0.306403316374161, 'w5': 0.8324062715198399, 'w6': 0.08712508155932532, 'w7': 0.40710241177566864, 'w8': 0.00333018915029325, 'w9': 0.491656926097263, 'w10': 0.9186238379520102, 'w11': 0.23096091208816233, 'w12': 0.12776326446621356, 'w13': 0.10845934357626283, 'w14': 0.03552126416868338}. 

[I 2020-10-06 08:11:57,269] Trial 945 finished with value: 0.014473211231975123 and parameters: {'w1': 0.42046689105727936, 'w2': 0.33133631072045633, 'w3': 0.10692934553790395, 'w4': 0.2926511937299943, 'w5': 0.8676488137844568, 'w6': 0.08394644524026484, 'w7': 0.43424714832798, 'w8': 0.0036237355178212103, 'w9': 0.47719097345482303, 'w10': 0.9177835894541269, 'w11': 0.2216213609735753, 'w12': 0.0974438618583695, 'w13': 0.10585016643610853, 'w14': 0.04131468853757335}. Best is trial 921 with value: 0.014472813578099684.
[I 2020-10-06 08:11:57,399] Trial 946 finished with value: 0.014472970885226982 and parameters: {'w1': 0.44309930560417904, 'w2': 0.2945283523248961, 'w3': 0.10604274992378836, 'w4': 0.2996780378534042, 'w5': 0.836992223596482, 'w6': 0.0819663924579587, 'w7': 0.43018201649866883, 'w8': 0.0003310046896632113, 'w9': 0.47773376937993245, 'w10': 0.918923263014366, 'w11': 0.23616023008741147, 'w12': 0.13735037107488157, 'w13': 0.12057012918348169, 'w14': 0.04443878141847106

[I 2020-10-06 08:11:59,270] Trial 960 finished with value: 0.014473267239869159 and parameters: {'w1': 0.38542887232361345, 'w2': 0.3457441057763427, 'w3': 0.09936981199002734, 'w4': 0.29093045429645037, 'w5': 0.8871170709261477, 'w6': 0.07771259967030555, 'w7': 0.4463772225001985, 'w8': 0.0008596582651686313, 'w9': 0.43685433548213276, 'w10': 0.9288464052577247, 'w11': 0.2087350405843699, 'w12': 0.06322932671378467, 'w13': 0.12747975738086686, 'w14': 0.06978178691888744}. Best is trial 921 with value: 0.014472813578099684.
[I 2020-10-06 08:11:59,405] Trial 961 finished with value: 0.014473021620562077 and parameters: {'w1': 0.4200997404644076, 'w2': 0.3004899058267105, 'w3': 0.09869177260112247, 'w4': 0.28888632994200714, 'w5': 0.8657080848518378, 'w6': 0.0721830520478298, 'w7': 0.4454071549358157, 'w8': 0.0028979949143241018, 'w9': 0.452544808167987, 'w10': 0.9312297983300019, 'w11': 0.22773489557797733, 'w12': 0.14237845680045105, 'w13': 0.1077368973268424, 'w14': 0.0714082480990226

[I 2020-10-06 08:12:01,288] Trial 975 finished with value: 0.014472832413655752 and parameters: {'w1': 0.4297510084952737, 'w2': 0.29291219954402653, 'w3': 0.13026559684757488, 'w4': 0.30492180138816766, 'w5': 0.8422031602533886, 'w6': 0.06528199544675849, 'w7': 0.4305490849504295, 'w8': 0.003459223860664747, 'w9': 0.45592092123756267, 'w10': 0.9403656559297685, 'w11': 0.2036471352201035, 'w12': 0.11937270365618413, 'w13': 0.1373375188943577, 'w14': 0.06389766511493866}. Best is trial 921 with value: 0.014472813578099684.
[I 2020-10-06 08:12:01,426] Trial 976 finished with value: 0.014473094141339956 and parameters: {'w1': 0.39180844701991274, 'w2': 0.29090337940162325, 'w3': 0.12668436194619856, 'w4': 0.2826704511274121, 'w5': 0.8389219043503601, 'w6': 0.06154519022461805, 'w7': 0.4246391705104732, 'w8': 0.0020918161164269226, 'w9': 0.4536589492283279, 'w10': 0.9357024548041873, 'w11': 0.19707969467631206, 'w12': 0.09569929927640401, 'w13': 0.10062950529835518, 'w14': 0.06780327433210

[I 2020-10-06 08:12:03,602] Trial 990 finished with value: 0.014473289087902465 and parameters: {'w1': 0.42295632594478905, 'w2': 0.30375143768249124, 'w3': 0.08995948692409828, 'w4': 0.24413199709388675, 'w5': 0.8832313248089695, 'w6': 0.07036113951893924, 'w7': 0.4543117427796682, 'w8': 0.00038887517677362797, 'w9': 0.444794566688889, 'w10': 0.954645132993984, 'w11': 0.21000185368648722, 'w12': 0.0474292858440134, 'w13': 0.11818409766943572, 'w14': 0.08298359190674125}. Best is trial 921 with value: 0.014472813578099684.
[I 2020-10-06 08:12:03,746] Trial 991 finished with value: 0.014474925096899672 and parameters: {'w1': 0.39551629786981113, 'w2': 0.2859250275296973, 'w3': 0.11437909322830725, 'w4': 0.0033557751591693497, 'w5': 0.844097556254483, 'w6': 0.07519815481467303, 'w7': 0.41650715546096484, 'w8': 0.02197764433210339, 'w9': 0.4111904802935312, 'w10': 0.9315649415483045, 'w11': 0.19134209883668865, 'w12': 0.1053737434780737, 'w13': 0.14136369576879387, 'w14': 0.05562286314938

In [103]:
best_trial = study.best_params
best_trial

{'w1': 0.4231697378311346,
 'w2': 0.30662539572939684,
 'w3': 0.11894711405463247,
 'w4': 0.3226078598379309,
 'w5': 0.801624490929455,
 'w6': 0.08758073646745682,
 'w7': 0.40649433082926245,
 'w8': 0.002406164383276838,
 'w9': 0.4968407587960485,
 'w10': 0.9081185529458068,
 'w11': 0.15984965691635994,
 'w12': 0.15668733760899847,
 'w13': 0.0916132832704552,
 'w14': 0.023923426397240113}

In [104]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

blend_boi = blend(all_preds)
noice_weights = np.array(list(best_trial.values()))
sample_infer = blend_boi.predict(noice_weights)
loss = log_loss_metric(y_val, sample_infer)
print("loss on hold set: ", loss)  ## scalar only on model 1 was 0.014533262113321026

loss on hold set:  0.014472813578099684


In [ ]:
list(best_trial.values())

In [ ]:
df_w = pd.DataFrame(
    np.array(list(best_trial.values())).reshape(1,-1), 
    columns = list(best_trial.keys())
)

In [ ]:
df_w.to_csv("blend_weights.csv", index = False)

In [ ]:
df_w